<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/MissingImputation/MissForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Run on Python3.8**

In [ ]:
!pip install missingpy
!pip install scikit-learn==1.1.2

In [ ]:
import sklearn
print(sklearn.__version__)

1.1.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Wide Table/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,Recent_Creatinine,Recent_eGFR,Recent_Date,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,...,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC,LabLocation,LabCwhen
0,924852,2.63,15.8,2560-08-07 12:32:50.527,3349148,2.754717,0,30,0,30,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-01 08:15:57.943
1,989278,2.03,32.3,2560-08-11 08:22:48.737,3348711,-46.720000,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,80.3,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-01 09:14:12.527
2,2103792,1.05,55.9,2560-08-14 14:38:44.150,3369057,11.018868,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,44.7,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-08 09:42:53.923
3,953611,1.03,51.1,2560-08-15 02:05:01.323,3366405,17.222520,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,33.5,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-06 10:35:19.840
4,488150,1.94,32.6,2560-08-15 09:18:43.240,3356167,-79.002653,0,0,0,0,...,NaN,17.0,NaN,NaN,NaN,NaN,114.2,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร,2559-08-03 08:44:33.377


In [ ]:
data = pd.DataFrame(df)

In [ ]:
import numpy as np

In [ ]:
columns_to_missfr = [col for col in data.columns
                     if (col not in ['Slopes', 'PatientUID', 'PatientVisitUID',
                                     'InPatient', 'OutPatient', 'Emergency',
                                     'eGFR_t1','time_t1', 'VisitLocation', 'StartDTTM',
                                     'EndDttm', 'LabCwhen'])]

data_selected = data[columns_to_missfr].copy()


In [ ]:
data_selected

,Recent_Creatinine,Recent_eGFR,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,G1M1:ACEI,...,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs),L1:eGFR,L14:WBC,LabLocation
0,2.63,15.80,0,30,0,30,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.00,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
1,2.03,32.30,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.30,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
2,1.05,55.90,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.70,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
3,1.03,51.10,0,0,0,0,100,0,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.50,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
4,1.94,32.60,0,0,0,0,0,0,0,0,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,114.20,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,1.30,55.28,0,90,0,0,0,0,0,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.30,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52616,1.30,55.28,0,90,0,90,0,0,0,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.10,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52617,1.30,55.28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.60,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
52618,1.30,55.28,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.06,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร


In [ ]:
for col in data_selected.columns:
  print(col, "\t", data_selected[col].dtypes, "\t", np.sum(data_selected[col].isna()))

Recent_Creatinine 	 float64 	 0
Recent_eGFR 	 float64 	 0
G1M2:ARB 	 int64 	 0
G3M1:Statin 	 int64 	 0
G5M2:NSAIDs 	 int64 	 0
G1M5:Diuretic 	 int64 	 0
G1M7:Beta blocker 	 int64 	 0
G1M3:Dihydropyridine CCB 	 int64 	 0
G1M6:MRA 	 int64 	 0
G1M1:ACEI 	 int64 	 0
G2M2:Metformin 	 int64 	 0
G2M3:DPP4 inhibitor 	 int64 	 0
G2M1:SU 	 int64 	 0
G3M3:Fibrate 	 int64 	 0
G2M5:SGLT2i 	 int64 	 0
G4M1:Allopurinol 	 int64 	 0
G1M9:Vasodilator 	 int64 	 0
G1M8:Alpha blocker 	 int64 	 0
G2M6:Insulin 	 int64 	 0
G2M4:TZD 	 int64 	 0
G4M2:Febuxostat 	 int64 	 0
G3M2:Ezetimibe 	 int64 	 0
G1M4:Non-dihydropyridine CCB 	 int64 	 0
G5M1:Ketoanalog amino acid 	 int64 	 0
G2M7:GLP1-RA 	 int64 	 0
G4M3:Uricosuric agent 	 int64 	 0
Sex 	 object 	 0
Age 	 int64 	 0
NATNL 	 object 	 0
Systolic BP 	 float64 	 4690
Diastolic BP 	 float64 	 4712
BMI 	 float64 	 15016
BSA 	 float64 	 15034
D1:Hypertension 	 int64 	 0
D2:Diabetes mellitus 	 int64 	 0
D3:Dyslipidemia 	 int64 	 0
D4:SLE 	 int64 	 0
D5:Systemic scler

In [ ]:
import re
original_columns = data_selected.columns.tolist()
data_selected = data_selected.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '-', x))
data_selected.head()

,Recent_Creatinine,Recent_eGFR,G1M2-ARB,G3M1-Statin,G5M2-NSAIDs,G1M5-Diuretic,G1M7-Beta-blocker,G1M3-Dihydropyridine-CCB,G1M6-MRA,G1M1-ACEI,...,L26-Vitamin-D,L27-UrineAlbumin-Cr-ratio,L28-UrineProtein-Cr-ratio,L29-UrineVolume-24hrs-,L30-UrineUrea-24hrs-,L31-UrineSodium-24hrs-,L32-UrineProtein-24hrs-,L1-eGFR,L14-WBC,LabLocation
0,2.63,15.8,0,30,0,30,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
1,2.03,32.3,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.3,5.2,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
2,1.05,55.9,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.7,7.4,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร
3,1.03,51.1,0,0,0,0,100,0,0,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.5,NaN,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย
4,1.94,32.6,0,0,0,0,0,0,0,0,...,NaN,NaN,17.0,NaN,NaN,NaN,NaN,114.2,4.1,ห้องปฏิบัติการเคมีคลินิก เวชศาสตร์ชันสูตร


In [ ]:
data_selected['Sex'] = data_selected['Sex'].replace({'ชาย': 0, 'หญิง': 1})

In [ ]:
nationality_map = {'ไทย': 0}

caucasian = ['บริติช  (อังกฤษ, สก็อตแลนด์)', 'อเมริกัน', 'เยอรมัน', 'โอมาน', 'ออสเตรเลีย', 'แคนาดา',
             'อิตาลี', 'สวิส', 'ซีเรีย', 'เดนมาร์ก', 'นอร์เวย์', 'ดัตช์', 'สเปน', 'เบลเยียม', 'ไอซ์แลนด์',
             'ฝรั่งเศส', 'เซนต์คิตส์และเนวิส', 'สวีเดน', 'ออสเตรีย', 'ไซปรัส']
asian = ['ลาว','จีน','เวียดนาม','เขมรอพยพ','ไทยลื้อ','ญี่ปุ่น', 'จีน(ไต้หวัน)']
other = ['ไม่ระบุ','อื่นๆ','ไร้สัญชาติ']

for nationality in asian:
    nationality_map[nationality] = 1

for nationality in caucasian:
    nationality_map[nationality] = 2

for nationality in other:
    nationality_map[nationality] = 3

data_selected['NATNL'] = data_selected['NATNL'].replace(nationality_map)

In [ ]:
data_selected['LabLocation'] = np.where(data_selected['LabLocation'] == "ห้องปฏิบัติการจุลทรรศน์วินิจฉัย", 1, 0)
data_selected['LabLocation'].value_counts()

1    26916
0    25704
Name: LabLocation, dtype: int64

In [ ]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [ ]:
imputer = MissForest(random_state=0, verbose=1)
X = data_selected.copy()
X_imputed = imputer.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
/usr/l

In [ ]:
data_missfr = pd.DataFrame(X_imputed, columns=X.columns)
data_missfr

In [ ]:
data_missfr = pd.to_excel('Missfr.xlsx')

#**Get data to contunue**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Wide Table/WideTable.xlsx')
df = df.iloc[:, 1:]
df.head()

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L20:HbA1c,L21:Cholesterol,L22:Triglyceride,L23:LDL Cholesterol,L24:HDL Cholesterol,L28:UrineProtein/Cr ratio,eGFR_t1,time_t1,LabLocation,LabCwhen
0,2299,3386255,-7.158596,90,0,0,0,0,0,0,...,NaN,215.0,81.0,123.0,99.0,NaN,63.60,2017-10-03 09:19:54.710,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2559-08-15 09:51:55.203
1,2299,4546471,5.550691,90,90,0,0,0,0,0,...,NaN,219.0,64.0,146.0,73.0,NaN,70.20,2018-12-12 09:18:49.913,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2560-10-03 09:19:54.920
2,2299,5812902,-6.063710,100,100,34,0,0,0,0,...,5.3,193.0,62.0,129.0,65.0,NaN,64.02,2019-12-19 09:51:24.427,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2561-12-12 10:15:36.930
3,2299,6947220,-12.812245,35,0,1,35,0,0,0,...,NaN,205.0,69.0,139.0,76.0,NaN,48.54,2021-03-04 10:01:42.113,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2562-12-19 09:51:24.577
4,2299,8186767,28.287500,100,0,15,0,0,0,0,...,5.5,198.0,74.0,129.0,60.0,NaN,80.47,2022-04-21 08:45:54.580,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2564-03-04 10:07:37.037


In [4]:
import pandas as pd
data = pd.read_excel("/content/drive/MyDrive/ckdresearch2023/Assign/Assign 6: MissingValue/Python/Missfr.xlsx")
data.head()

,G1M2-ARB,G3M1-Statin,G5M2-NSAIDs,G1M5-Diuretic,G1M7-Beta-blocker,G1M3-Dihydropyridine-CCB,G1M6-MRA,G1M1-ACEI,G2M2-Metformin,G2M3-DPP4-inhibitor,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,90,0,0,0,0,0,0,0,0,0,...,5.8998,256.67,91.0,5.559,215.0,81.0,123.0,99.0,65.810,1
1,90,90,0,0,0,0,0,0,0,0,...,5.6377,260.39,86.0,5.489,219.0,64.0,146.0,73.0,105.881,1
2,100,100,34,0,0,0,0,0,0,0,...,9.4100,313.00,100.0,5.300,193.0,62.0,129.0,65.0,51.688,1
3,35,0,1,35,0,0,0,0,0,0,...,4.9215,206.32,89.0,5.414,205.0,69.0,139.0,76.0,20.270,1
4,100,0,15,0,0,0,0,0,0,0,...,5.4200,434.00,88.0,5.500,198.0,74.0,129.0,60.0,47.001,1


In [6]:
import numpy as np
columns_to_keep  = df[['PatientUID', 'PatientVisitUID', 'Slopes','InPatient', 'OutPatient', 'Emergency','time_t1','eGFR_t1', 'VisitLocation', 'StartDTTM','EndDttm', 'LabCwhen']]
result = pd.concat([columns_to_keep, data], axis=1)
result.head()

,PatientUID,PatientVisitUID,Slopes,InPatient,OutPatient,Emergency,time_t1,eGFR_t1,VisitLocation,StartDTTM,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,2299,3386255,-7.158596,0,1,0,2017-10-03 09:19:54.710,63.60,เวชปฏิบัติทั่วไป (OPD)-D03,2559-08-15 08:13:26.043,...,5.8998,256.67,91.0,5.559,215.0,81.0,123.0,99.0,65.810,1
1,2299,4546471,5.550691,0,1,0,2018-12-12 09:18:49.913,70.20,เวชปฏิบัติทั่วไป (OPD)-D03,2560-10-03 07:37:22.777,...,5.6377,260.39,86.0,5.489,219.0,64.0,146.0,73.0,105.881,1
2,2299,5812902,-6.063710,0,1,0,2019-12-19 09:51:24.427,64.02,เวชปฏิบัติทั่วไป (OPD)-D03,2561-12-12 07:58:57.190,...,9.4100,313.00,100.0,5.300,193.0,62.0,129.0,65.0,51.688,1
3,2299,6947220,-12.812245,0,1,0,2021-03-04 10:01:42.113,48.54,เวชปฏิบัติทั่วไป (OPD)-D03,2562-12-19 08:19:38.647,...,4.9215,206.32,89.0,5.414,205.0,69.0,139.0,76.0,20.270,1
4,2299,8186767,28.287500,0,1,0,2022-04-21 08:45:54.580,80.47,เวชปฏิบัติทั่วไป (OPD)-D03,2564-03-04 07:55:40.703,...,5.4200,434.00,88.0,5.500,198.0,74.0,129.0,60.0,47.001,1


In [7]:
columns_with_null  = result.isnull().any()
if columns_with_null.any():
    print("There are columns with null values.")
else:
    print("No columns have null values.")

No columns have null values.


> แปลง LabCwhen เป็นคศ

In [12]:

import pandas as pd

df1 = pd.DataFrame(result)

# Splitting the 'time' column into separate columns
df1[['date', 'timestamp']] = df1['LabCwhen'].str.split(' ', expand=True)
df1[['year', 'month', 'day']] = df1['date'].str.split('-', expand=True)
df1[['hour', 'minute', 'second']] = df1['timestamp'].str.split(':', expand=True)
df1[['second', 'nanosecond']] = df1['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df1['year'] = df1['year'].astype(int) - 543

# Combine columns into a datetime column
df1['date'] = pd.to_datetime(df1[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df1['timestamp'] = pd.to_timedelta(df1['timestamp'])

# Convert date to datetime
df1['date'] = pd.to_datetime(df1['date'])

# Merge timestamp and date columns
df1['LabCwhen'] = df1['date'] + df1['timestamp']

In [17]:
columns_to_drop = ['date', 'timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second', 'nanosecond']
df1 = df1.drop(columns=columns_to_drop, axis=1)
df1

,PatientUID,PatientVisitUID,Slopes,InPatient,OutPatient,Emergency,time_t1,eGFR_t1,VisitLocation,StartDTTM,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,2299,3386255,-7.158596,0,1,0,2017-10-03 09:19:54.710,63.60,เวชปฏิบัติทั่วไป (OPD)-D03,2559-08-15 08:13:26.043,...,5.8998,256.67,91.00,5.559,215.0,81.00,123.00,99.00,65.810,1
1,2299,4546471,5.550691,0,1,0,2018-12-12 09:18:49.913,70.20,เวชปฏิบัติทั่วไป (OPD)-D03,2560-10-03 07:37:22.777,...,5.6377,260.39,86.00,5.489,219.0,64.00,146.00,73.00,105.881,1
2,2299,5812902,-6.063710,0,1,0,2019-12-19 09:51:24.427,64.02,เวชปฏิบัติทั่วไป (OPD)-D03,2561-12-12 07:58:57.190,...,9.4100,313.00,100.00,5.300,193.0,62.00,129.00,65.00,51.688,1
3,2299,6947220,-12.812245,0,1,0,2021-03-04 10:01:42.113,48.54,เวชปฏิบัติทั่วไป (OPD)-D03,2562-12-19 08:19:38.647,...,4.9215,206.32,89.00,5.414,205.0,69.00,139.00,76.00,20.270,1
4,2299,8186767,28.287500,0,1,0,2022-04-21 08:45:54.580,80.47,เวชปฏิบัติทั่วไป (OPD)-D03,2564-03-04 07:55:40.703,...,5.4200,434.00,88.00,5.500,198.0,74.00,129.00,60.00,47.001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,3319961,9270330,-4.256753,0,1,0,2023-04-04 08:28:58.833,33.68,บูรณาการนอกเวลา (คลินิก)-OD191,2565-03-14 14:31:31.910,...,10.3200,306.00,103.00,5.900,165.0,311.00,84.00,43.00,131.300,1
52616,3319971,9270331,-0.692078,0,1,0,2023-04-04 08:49:20.047,26.47,บูรณาการนอกเวลา (คลินิก)-OD191,2565-03-14 14:31:53.720,...,9.5600,361.00,99.00,7.100,154.0,196.00,83.00,42.00,325.800,1
52617,3320048,9242721,7.217875,0,0,1,2023-04-10 08:36:15.217,70.36,OPD AE-DV191,2565-03-05 12:36:33.263,...,4.6100,131.00,123.34,6.511,112.0,96.00,64.00,33.00,63.789,1
52618,3320050,9257281,-32.794557,0,1,0,2023-04-10 08:58:57.267,30.69,หน่วยคลินิกเฉพาะทางศูนย์บริการทางการแพทย์ชั้นเลิศ,2565-03-10 07:42:44.243,...,9.4400,305.00,131.06,6.871,206.4,152.73,149.15,44.13,181.380,1


In [18]:
df1.to_excel('MissForest.xlsx')